In [1]:
# Main functions
import networkx as nx
import community
import random as rdm
import numpy as np
from numpy.random import choice
import dit
import queue 
from dit.divergences import jensen_shannon_divergence
from scipy.stats import entropy
from numpy.linalg import norm
from scipy.sparse import linalg

# Utilities
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import time
import re
from os import listdir
from scipy.io import loadmat
from matplotlib2tikz import save as tikz_save
from matplotlib2tikz import get_tikz_code 

Load graph and attributes

In [2]:
# implement graph as networkX
# facebook100 graphs use Matlab matrix 
# others use txt files
def load_graph(path, graph_type):
    if graph_type == "Directed":
        graph = nx.DiGraph()
        with open(path) as f:
            lines = f.readlines()
            for line in lines:
                edge = line.strip().split()
                graph.add_edge(*edge)
        return graph
    elif graph_type == "Undirected":
        graph = nx.Graph()
        with open(path) as f:
            lines = f.readlines()
            for line in lines:
                edge = line.strip().split()
                graph.add_edge(*edge)
        return graph
    elif graph_type == "facebook100":
        adj_matrix = loadmat(path)['A'].toarray()
            # A -- adjacency matrix
            # local_info -- attributes
        graph = nx.from_numpy_matrix(adj_matrix)
        return graph
    else:
        print("graph type not recognized")

In [3]:
# build attribute list and bucket
# facebook100 graphs use Matlab matrix, they have at most 2 attributes for each nodes
# citation graph, wiki use one-to-one table of node attributes
# other ground-truth communities graphs use buckets without attributes' name
def build_attribute_list_and_bucket(path, graph_type):
    if graph_type == "facebook100":
        attribute_matrix = loadmat(path)['local_info']
        attribute_list = {}
        for i in range(len(attribute_matrix)):
            major = attribute_matrix[i][2]
            major_2 = attribute_matrix[i][3]
            year = attribute_matrix[i][5]       
            attribute_list[i] = (major, major_2, year)
        
        attribute_bucket = {}
        for k in attribute_list.keys():
            first_pair = (attribute_list[k][0], attribute_list[k][2])
            if first_pair in attribute_bucket:
                attribute_bucket[first_pair].append(k)
            else:
                attribute_bucket[first_pair] = [k]
            if attribute_list[k][1] != 0:
                second_pair = (attribute_list[k][1], attribute_list[k][2])
                if second_pair in attribute_bucket:
                    attribute_bucket[second_pair].append(k)
                else:
                    attribute_bucket[second_pair] = [k]
        return attribute_list, attribute_bucket

    
    elif graph_type == "cit-Patents":
        attribute_list = {}
        attribute_bucket = {}
        with open(path, 'r') as document:
            lines = document.readlines()
            for line in lines:
                instance = line.strip().split(',')
                attribute_list[instance[0]] = [instance[11]]
        for k in attribute_list.keys():
            if attribute_list[k][0] not in attribute_bucket:
                attribute_bucket[attribute_list[k][0]] = [k]
            else:
                attribute_bucket[attribute_list[k][0]].append(k)
        return attribute_list, attribute_bucket
    
    
    elif graph_type == "communities":
        attribute_list = {}
        attribute_bucket = {}
        with open(path, 'r') as document:
            bucket_index = 0
            lines = document.readlines()
            for line in lines:
                instance = line.strip().split('\t')
                attribute_bucket[bucket_index] = instance
                for i in instance:
                    if i not in attribute_list.keys():
                        attribute_list[i] = [bucket_index]
                    else:
                        attribute_list[i].append(bucket_index)
                bucket_index += 1
        return attribute_list, attribute_bucket
    
    elif graph_type == "wiki":
        attribute_list = {}
        attribute_bucket = {}
        with open(path, 'r') as document:
            lines = document.readlines()
            for line in lines:
                instance = line.strip().split(' ')
                attribute_bucket[instance[0]] = instance[1:]
                for i in instance[1:]:
                    if i not in attribute_list.keys():
                        attribute_list[i] = [instance[0]]
                    else:
                        attribute_list[i].append(instance[0])
        return attribute_list, attribute_bucket

In [4]:
# this is same as above but use fake attributes generated by Louvain algorithm
def generate_attribute_list_and_bucket(graph):
    if graph.is_directed():
        partition = community.best_partition(graph.to_undirected())
    else:
        partition = community.best_partition(graph)
    attribute_list = {}
    attribute_bucket = {}
    for i in partition.keys():
        attribute_list[i] = [partition[i]]
    for k in attribute_list.keys():
        if attribute_list[k][0] not in attribute_bucket:
            attribute_bucket[attribute_list[k][0]] = [k]
        else:
            attribute_bucket[attribute_list[k][0]].append(k)
    return attribute_list, attribute_bucket

Sampling Method

In [21]:
# calculate alpha based on percentage of nodes sampled from random walk
# calculate_alpha(g, ab, 1) = 0
def calculate_alpha(graph, attribute_list, attribute_bucket, random_walk_percentage):
    number_nodes = nx.number_of_nodes(graph)
    number_attributed_nodes = len(attribute_list)
    totalcount = 0
    for i in attribute_bucket.values():
        totalcount += len(i)**2
    total_attri = 0
    for i in attribute_list:
        total_attri+=len(i)
    prior_visit_attributed_nodes = number_attributed_nodes/number_nodes
    expectation_bucketnodes = totalcount*total_attri/(number_attributed_nodes*number_attributed_nodes)
    return ((1/random_walk_percentage)-1)/(prior_visit_attributed_nodes*expectation_bucketnodes)

In [22]:
# random node sampling
def random_node_sampling(graph, percentage, seed = None):
    # set random seed
    rdm.seed(seed)
    samplednodes = rdm.sample(graph.nodes(), k = round(nx.number_of_nodes(graph)*percentage))
    return graph.subgraph(samplednodes)

In [23]:
# random walk sampling
def random_walk_sampling(und_graph, sample_percentage, jump_iteration = 100, seed = None):
    # set random seed
    rdm.seed(seed)
    
    # sample size round down to interger
    sample_size = int(nx.number_of_nodes(und_graph) * sample_percentage)
    
    # set starting node
    startnode = rdm.sample(und_graph.nodes(),1)[0]
    currentnode = startnode
    
    # used for jump when no new node visited in certain iteration
    restart_iteration = 0 
    last_number_of_nodes = 0
    
    # result node set and total iteration
    nodelist = set()
    # visited_time = {}
    total_iteration = 0
    
    while len(nodelist) < sample_size:
        # add current node
        total_iteration += 1
        '''
        degree = und_graph.degree(currentnode)
        if degree not in visited_time:
            visited_time[degree] = 1
        else:
            visited_time[degree] += 1
        '''
        nodelist.add(currentnode)
        
        # move a step forward
        nextnode = rdm.sample(list(und_graph[currentnode]),1)[0]
        currentnode = nextnode
        
        # find a new startnode if number of nodes in sample does not grow
        if restart_iteration < jump_iteration:
            restart_iteration += 1
        else:
            if last_number_of_nodes == len(nodelist):
                startnode = rdm.sample(und_graph.nodes(),1)[0]
                currentnode = startnode
            restart_iteration = 0
            last_number_of_nodes = len(nodelist)
    return und_graph.subgraph(nodelist), total_iteration #, visited_time 

In [24]:
# random walk with restart sampling
def random_walk_with_restart_sampling(und_graph, sample_percentage, restart_prob, jump_iteration = 100, seed = None):
    # set random seed
    rdm.seed(seed)
    
    # sample size round down to interger
    sample_size = int(nx.number_of_nodes(und_graph) * sample_percentage)
    
    # set starting node
    startnode = rdm.sample(und_graph.nodes(),1)[0]
    currentnode = startnode
    
    # used for jump when no new node visited in certain iteration
    restart_iteration = 0 
    last_number_of_nodes = 0
    
    # result node set and total iteration
    nodelist = set()
    total_iteration = 0
    
    while len(nodelist) < sample_size:
        # add current node
        total_iteration += 1
        
        
        nodelist.add(currentnode)
        
        # restart with certain prob
        x = rdm.random()
        if x < restart_prob:
            currentnode = startnode
        else:    
            # move a step forward
            nextnode = rdm.sample(list(und_graph[currentnode]),1)[0]
            currentnode = nextnode
        
        # find a new startnode if number of nodes in sample does not grow
        if restart_iteration < jump_iteration:
            restart_iteration += 1
        else:
            if last_number_of_nodes == len(nodelist):
                startnode = rdm.sample(und_graph.nodes(),1)[0]
                currentnode = startnode
            restart_iteration = 0
            last_number_of_nodes = len(nodelist)
    return und_graph.subgraph(nodelist) ,total_iteration

In [25]:
# Forest Fire Sampling
def forest_fire_sampling(und_graph, sample_percentage, burn_prob, seed = None):
    # set random seed
    rdm.seed(seed)
    
    # sample size round down to interger
    sample_size = int(nx.number_of_nodes(und_graph) * sample_percentage)
    
    # set starting node
    startnode = rdm.sample(und_graph.nodes(),1)[0]
    currentnode = startnode
    explore_list = queue.Queue(maxsize = nx.number_of_nodes(und_graph))
    explore_list.put(currentnode)
    
    # result node set and total iteration
    nodelist = set()
    visitedlist = set()
    total_iteration = 0
    
    while len(nodelist) < sample_size:
        visitedlist.add(currentnode)
        total_iteration += 1
        
        # set new nodes in queue if empty
        if explore_list.empty():
            currentnode = rdm.sample(und_graph.nodes(),1)[0]
            if currentnode not in nodelist:
                explore_list.put(currentnode)
        else:
            # dequeue node and explore its neighbor
            currentnode = explore_list.get()
            nodelist.add(currentnode)
            for burn_node in list(und_graph[currentnode]):
                if burn_node not in visitedlist:
                    x = rdm.random()
                    if x < burn_prob:
                        explore_list.put(burn_node)
                        visitedlist.add(burn_node)
        
    return und_graph.subgraph(nodelist), total_iteration
        

In [26]:
# Forest Fire Sampling(Directed)
# backward_burn_prob = forward_burn_prob/backward_burn_prob_ratio (which is an interger in most cases)
def directed_forest_fire_sampling(di_graph, sample_percentage, forward_burn_prob, backward_burn_prob_ratio, seed = None):
    # set random seed
    rdm.seed(seed)
    
    # sample size round down to interger
    sample_size = int(nx.number_of_nodes(di_graph) * sample_percentage)
    backward_burn_prob = forward_burn_prob/backward_burn_prob_ratio
    
    # set starting node
    startnode = rdm.sample(di_graph.nodes(),1)[0]
    currentnode = startnode
    explore_list = queue.Queue(maxsize = nx.number_of_nodes(di_graph))
    explore_list.put(currentnode)
    
    # result node set and total iteration
    nodelist = set()
    visitedlist = set()
    total_iteration = 0
    
    while len(nodelist) < sample_size:
        visitedlist.add(currentnode)
        total_iteration += 1
        
        # set new nodes in queue if empty
        if explore_list.empty():
            currentnode = rdm.sample(di_graph.nodes(),1)[0]
            if currentnode not in nodelist:
                explore_list.put(currentnode)
        else:
            # dequeue node and explore its neighbor
            currentnode = explore_list.get()
            nodelist.add(currentnode)
            # forward burn
            for forward_burn_node in list(di_graph[currentnode]):
                if forward_burn_node not in visitedlist:
                    x = rdm.random()
                    if x < forward_burn_prob:
                        explore_list.put(forward_burn_node)
                        visitedlist.add(forward_burn_node)
            # backward burn            
            for backward_burn_node in list(di_graph.predecessors(currentnode)):
                if backward_burn_node not in visitedlist:
                    y = rdm.random()
                    if y < backward_burn_prob:
                        explore_list.put(backward_burn_node)
                        visitedlist.add(backward_burn_node)
        
    return di_graph.subgraph(nodelist) ,total_iteration

In [60]:
# powerlaw.pdf(x, k) = c * x**(-k) default k = 2
# 0<a<1 in degree distribution
# low_degree and high_degree are intergers
# return largest connected component
def powerlaw_degree_graph_generator(n, a, seed = None):
    rdm.seed(seed)
    r = nx.utils.powerlaw_sequence(n, exponent = a)
    int_r = list(map(int, r))
    if sum(int_r)%2 == 1:
        int_r[0] = int_r[0] + 1
    G = nx.configuration_model(int_r,create_using = nx.Graph(), seed = seed)
    cc_list = sorted(list(nx.connected_components(G)), key = len, reverse = True)
    Gc = G.subgraph(cc_list[0])
    return Gc

In [14]:
sub_g, iteration = Frontier_sampling(g, 0.1)

In [16]:
print(nx.number_of_nodes(sub_g), nx.number_of_edges(sub_g))

33486 77342


In [27]:
# Frontier Sampling
def Frontier_sampling(und_graph, sample_percentage, random_walker = 5, seed = None):
    # set random seed
    rdm.seed(seed)
    
    # sample size round down to interger
    sample_size = int(nx.number_of_nodes(und_graph) * sample_percentage)
    
    # set starting node
    current_walker = rdm.sample(und_graph.nodes(), random_walker)
    # initialize degrees
    degree_list = [und_graph.degree[n] for n in current_walker]
    
    # result node set and total iteration
    nodelist = set()
    total_iteration = 0
    for cn in current_walker:
        nodelist.add(cn)
    
    while len(nodelist) < sample_size:
        # add current node
        total_iteration += 1
        currentnode, index = Frontier_sampling_select_node(current_walker, degree_list) # select walker dependently
        
        # move a step forward
        nextnode = rdm.sample(list(und_graph[currentnode]),1)[0]
        nodelist.add(nextnode)
        current_walker[index] = nextnode
        degree_list[index] = und_graph.degree[nextnode]
    
    return und_graph.subgraph(nodelist), total_iteration #, visited_time 

def Frontier_sampling_select_node(current_walker, degree_list):
    probability_distribution = [d/sum(degree_list) for d in degree_list]
    select_node = choice(current_walker, 1, p = probability_distribution)[0]
    return select_node, current_walker.index(select_node)

In [28]:
# Metropolis–Hastings random walk sampling
def MH_random_walk_sampling(und_graph, sample_percentage, jump_iteration = 100, seed = None):
    # set random seed
    rdm.seed(seed)
    
    # sample size round down to interger
    sample_size = int(nx.number_of_nodes(und_graph) * sample_percentage)
    
    # set starting node
    startnode = rdm.sample(und_graph.nodes(),1)[0]
    currentnode = startnode
    
    # used for jump when no new node visited in certain iteration
    restart_iteration = 0 
    last_number_of_nodes = 0
    
    # result node set and total iteration
    nodelist = set()
    total_iteration = 0
    
    while len(nodelist) < sample_size:
        # add current node
        total_iteration += 1
        nodelist.add(currentnode)        
        
        # MHRW process
        random_number = rdm.random()
        neighbors = list(und_graph[currentnode])
        list_of_candidates = neighbors[:]
        list_of_candidates.append(currentnode)
        probability_distribution = []        
        for n in neighbors:
            p_n = 1/und_graph.degree[currentnode] * min(1, und_graph.degree[currentnode]/und_graph.degree[n])
            probability_distribution.append(p_n)
        total = sum(probability_distribution)
        if total > 1:
            total = 1
        probability_distribution.append(1 - total)
        currentnode = choice(list_of_candidates, 1, p = probability_distribution)[0]
        
        # find a new startnode if number of nodes in sample does not grow
        if restart_iteration < jump_iteration:
            restart_iteration += 1
        else:
            if last_number_of_nodes == len(nodelist):
                startnode = rdm.sample(und_graph.nodes(),1)[0]
                currentnode = startnode
            restart_iteration = 0
            last_number_of_nodes = len(nodelist)
    return und_graph.subgraph(nodelist) ,total_iteration

In [29]:
# Metropolis–Hastings random walk sampling
def MH_random_walk_sampling_di(di_graph, sample_percentage, jump_iteration = 100, seed = None):
    # transfer graph to undirected
    und_graph = di_graph.to_undirected()
    
    # set random seed
    rdm.seed(seed)
    
    # sample size round down to interger
    sample_size = int(nx.number_of_nodes(und_graph) * sample_percentage)
    
    # set starting node
    startnode = rdm.sample(und_graph.nodes(),1)[0]
    currentnode = startnode
    
    # used for jump when no new node visited in certain iteration
    restart_iteration = 0 
    last_number_of_nodes = 0
    
    # result node set and total iteration
    nodelist = set()
    total_iteration = 0
    
    while len(nodelist) < sample_size:
        # add current node
        total_iteration += 1
        nodelist.add(currentnode)        
        
        # MHRW process
        random_number = rdm.random()
        neighbors = list(und_graph[currentnode])
        list_of_candidates = neighbors[:]
        list_of_candidates.append(currentnode)
        probability_distribution = []        
        for n in neighbors:
            p_n = 1/und_graph.degree[currentnode] * min(1, und_graph.degree[currentnode]/und_graph.degree[n])
            probability_distribution.append(p_n)
        total = sum(probability_distribution)
        if total > 1:
            total = 1
        probability_distribution.append(1 - total)
        currentnode = choice(list_of_candidates, 1, p = probability_distribution)[0]
        
        # find a new startnode if number of nodes in sample does not grow
        if restart_iteration < jump_iteration:
            restart_iteration += 1
        else:
            if last_number_of_nodes == len(nodelist):
                startnode = rdm.sample(und_graph.nodes(),1)[0]
                currentnode = startnode
            restart_iteration = 0
            last_number_of_nodes = len(nodelist)
    return di_graph.subgraph(nodelist) ,total_iteration

In [30]:
# Random walk + attributes sampling
# alpha from (0,1) uniform probability that node being selected from attribute bucket
def random_walk_attribute_sampling(graph, percentage, attribute_list, attribute_bucket, alpha, jump_iteration = 100, seed = None):
    # set random seed
    rdm.seed(seed)
    
    sample_size = round(nx.number_of_nodes (graph)*percentage)
    visitednodes = set()
    samplednodes = set()
    Gc = set()
    
    startnode = rdm.sample(graph.nodes(),1)[0]
    currentnode = startnode
    iteration = 0 
    total_iteration = 0
    sample_full = False
    last_number_of_nodes = 0
    
    while not sample_full:
        total_iteration += 1
        if currentnode not in visitednodes:
            # add currentnode
            visitednodes.add(currentnode)
            samplednodes.add(currentnode)

            # count 1st components
            if len(samplednodes) > sample_size:
                G = graph.subgraph(samplednodes)
                #Gc = max(nx.weakly_connected_components(G), key=len)
                Gc = max(nx.connected_components(G), key=len)
                # decide whether terminate
                if len(Gc) >= sample_size:
                    sample_full = True
                else:
                    samplednodes = Gc

            # add same attribute nodes from bucket
            if (not sample_full) and (currentnode in attribute_list.keys()):
                bucketnodes = bucketsampling(currentnode, attribute_list, attribute_bucket, alpha)
                for bn in bucketnodes:                
                    samplednodes.add(bn)        
                    
                    
        # add edge when currentnode is not a sink (not needed for undirected graph)
        if len(list(graph[currentnode])) > 0:
            nextnode = rdm.sample(list(graph[currentnode]),1)[0]
            currentnode = nextnode
        # jump back to the start node when reach a sink    
        else:
            currentnode = startnode

        # find a new startnode if number of nodes in sample does not grow
        if iteration < jump_iteration:
            iteration += 1
        else:
            if last_number_of_nodes == len(visitednodes):
                startnode = rdm.sample(graph.nodes(),1)[0]
                currentnode = startnode
            iteration = 0
            last_number_of_nodes = len(visitednodes)
                        
    G = graph.subgraph(samplednodes)
    #Gc = max(nx.weakly_connected_components(G), key=len) 
    Gc = max(nx.connected_components(G), key=len)
    return G.subgraph(Gc), total_iteration

In [31]:
# Random attributes sampling
# Random select an attribute based on attribute distribution, sample from this bukcet with prob alpha
def random_attribute_sampling(graph, percentage, attribute_list, attribute_bucket, alpha):
    sample_size = round(nx.number_of_nodes (graph)*percentage)
    samplednodes = set()
    sample_full = False
    samplednodes_candidate = rdm.sample(graph.nodes(), k = round(nx.number_of_nodes(graph)*percentage))
    rdm.shuffle(samplednodes_candidate)
    i = 0
    
    while not sample_full:
        currentnode = samplednodes_candidate[i]
        i+=1
        if currentnode in attribute_list.keys():
            bucketnodes = bucketsampling(currentnode, attribute_list, attribute_bucket, alpha)
            for bn in bucketnodes:
                if len(samplednodes) < sample_size:
                    samplednodes.add(bn)
                else:
                    sample_full = True
        else:
            if len(samplednodes) < sample_size:
                samplednodes.add(currentnode)
            else:
                sample_full = True
                
    return graph.subgraph(samplednodes)

In [32]:
def bucketsampling(node, attribute_list, attribute_bucket, alpha):
    samplednode = []
    for i in attribute_list[node]:
        for n in attribute_bucket[i]:
            random_number = rdm.random()
            if random_number <= alpha:
                samplednode.append(n)
    rdm.shuffle(samplednode)
    return samplednode

In [15]:
# Utility function used by random_walk_attribute_sampling
def bucketsampling_facebook(node, attribute_list, attribute_bucket, alpha):
    samplednode = []
    first_pair = (attribute_list[node][0], attribute_list[node][2])
    for node in attribute_bucket[first_pair]:
        random_number = rdm.random()
        if random_number <= alpha:
            samplednode.append(node)
    if attribute_list[node][1] != 0:
        second_pair = (attribute_list[node][1], attribute_list[node][2])
        for node in attribute_bucket[second_pair]:
            random_number = rdm.random()
            if random_number <= alpha:
                samplednode.append(node)
    rdm.shuffle(samplednode)
    return samplednode

Evaluations

Assortativity

In [16]:
# measuring assortativity
def assortativity(graph, attibute_list):
    nx.set_node_attributes(graph, attibute_list, 'attribute')
    ast = nx.attribute_assortativity_coefficient(graph, 'attribute')
    return ast

In [17]:
# manipulate al for facebook-100
def fitst_attribute(attribute_list):
    new_al = {}
    for i in attribute_list.keys():
        new_al[i] = (attribute_list[i][0], attribute_list[i][2])
    return new_al

Degree distribution

In [33]:
# degree distribution generalized by number of nodes
def degree_distribution(graph):
    degrees = {}
    generalizer = 1/nx.number_of_nodes (graph)
    for n in graph.nodes():
        degree = graph.degree(n)
        if degree not in degrees:
            degrees[degree] = 0
        degrees[degree] += generalizer
    return dict(sorted(degrees.items()))   

In [34]:
# outdegree distribution 
def outdegree_distribution(digraph):
    degrees = {}
    generalizer = 1/nx.number_of_nodes (digraph)
    for n in digraph.nodes():
        degree = digraph.out_degree(n)
        if degree not in degrees:
            degrees[degree] = 0
        degrees[degree] += generalizer
    return dict(sorted(degrees.items()))   

In [35]:
# outdegree distribution 
def indegree_distribution(digraph):
    degrees = {}
    generalizer = 1/nx.number_of_nodes (digraph)
    for n in digraph.nodes():
        degree = digraph.in_degree(n)
        if degree not in degrees:
            degrees[degree] = 0
        degrees[degree] += generalizer
    return dict(sorted(degrees.items()))  

Connected components

In [36]:
# CC distribution 
def cc_distribution(graph):
    cc_list = sorted(list(nx.connected_components(graph)), key = len, reverse=True)
    sizes = {}
    total = nx.number_of_nodes(graph)
    rank = 1
    for n in cc_list:
        sizes[rank] = len(n)/total
        rank += 1
    return dict(list(sizes.items()))  

In [37]:
# SCC distribution 
def scc_distribution(digraph):
    scc_list = sorted(list(nx.strongly_connected_components(digraph)), key = len, reverse=True)
    sizes = {}
    total = nx.number_of_nodes(digraph)
    rank = 1
    for n in scc_list:
        sizes[rank] = len(n)/total
        rank += 1
    return dict(list(sizes.items()))

In [38]:
# WCC distribution 
def wcc_distribution(digraph):
    wcc_list = sorted(list(nx.weakly_connected_components(digraph)), key = len, reverse=True)
    sizes = {}
    total = nx.number_of_nodes(digraph)
    rank = 1
    for n in wcc_list:
        sizes[rank] = len(n)/total
        rank += 1
    return dict(list(sizes.items()))

Clustering coefficient

In [39]:
# Clustering coefficient distribution
def clustering_coefficient_distribution(graph):
    clust_coefficients = nx.clustering(graph)
    coeff = {}
    count = {}
    for n in graph:
        degree = graph.degree(n)
        coefficient = clust_coefficients[n]
        if degree not in coeff:
            coeff[degree] = 0
            count[degree] = 0
        coeff[degree] += coefficient
        count[degree] += 1
    for key in coeff:
        coeff[key] = coeff[key]/count[key]
    return dict(sorted(coeff.items()))

In [40]:
# average absolute distance
def average_absolute_distance(cc1, cc2):
    result = []
    for i in cc1.keys():
        if i in cc2.keys():
            result.append(abs(cc1[i]-cc2[i]))
    return sum(result)/len(result)

In [41]:
# Transfer directed graph to undirected
def to_undirected(digraph):
    return digraph.to_undirected()

Min-Distance distribution

In [80]:
'''
# Min-Distance distribution
# This part was disabled, we use ANF to approximate due to size of the graphs
# We directly read from files, since snap.py package only supports python 2.7

def min_distance_distribution(graph):
    min_distance = {}
    for n in graph.nodes():
        start_time = time.time()
        length = nx.single_source_dijkstra_path_length(graph, n)
        for p in length:
            if p not in min_distance:
                min_distance[p] = 0
            min_distance[p] += 1
        #print("--- %s seconds ---" % (time.time() - start_time))
    total = sum(min_distance.values())

    for k in min_distance:
        min_distance[k] = min_distance[k]/total
    return sorted(min_distance.items())
'''

'\n# Min-Distance distribution\n# This part was disabled, we use ANF to approximate due to size of the graphs\n# We directly read from files, since snap.py package only supports python 2.7\n\ndef min_distance_distribution(graph):\n    min_distance = {}\n    for n in graph.nodes():\n        start_time = time.time()\n        length = nx.single_source_dijkstra_path_length(graph, n)\n        for p in length:\n            if p not in min_distance:\n                min_distance[p] = 0\n            min_distance[p] += 1\n        #print("--- %s seconds ---" % (time.time() - start_time))\n    total = sum(min_distance.values())\n\n    for k in min_distance:\n        min_distance[k] = min_distance[k]/total\n    return sorted(min_distance.items())\n'

In [81]:
def read_plothop_distribution(filename):    
    plothop = load_distribution(filename)
    min_distance = {}
    min_distance[0] = plothop[0]
    for i in plothop.keys():
        if i > 0:
            min_distance[i] = plothop[i] - plothop[i-1]
    for i in min_distance.keys():
        min_distance[i] /= plothop[len(plothop) - 1]
    return min_distance

SVD

In [29]:
# First left singular vector and singular values distribution
def get_SVD_distribution(graph, rank):
    start_time = time.time()

    A = nx.adj_matrix(graph).asfptype()
    U, s, Vh = linalg.svds(A, k = rank)
    print("--- %s seconds ---" % (time.time() - start_time))
    s_sorted = np.flipud(s)
    firstU_sorted = np.flipud(np.sort(np.absolute(U[:,0])))

    FLSV = {}
    SV = {}
    
    for i in range(len(s_sorted)):
        SV[i] = s_sorted[i]
    for i in range(len(firstU_sorted)):
        FLSV[i] = firstU_sorted[i]
        
    total = sum(SV.values())
    for i in SV.keys():
        SV[i] /= total

    return dict(list(FLSV.items())), dict(list(SV.items()))

In [30]:
# scale make it compareable
def sample_FLSV_distribution(flsv, percentage):
    new_flsv = {}
    skip = int(1/percentage)
    for i in range(round(len(flsv)*percentage)):
        new_flsv[i] = flsv[i*skip]
    total = sum(new_flsv.values())
    for i in new_flsv.keys():
        new_flsv[i] /= total
    return new_flsv

Measuring Attributes Cover Rate

In [31]:
# nodes form selected attribute/ all nodes
def attribute_cover_rate(subgraph, attribute_list, attribute_bucket):
    sampled_attributes = set()
    for i in subgraph.nodes():
        if i in attribute_list:
            for j in attribute_list[i]:
                sampled_attributes.add(j)
    sampled_attributed_nodes = set()
    for i in sampled_attributes:
        for j in attribute_bucket[i]:
            sampled_attributed_nodes.add(j)
    rate = len(sampled_attributed_nodes)/len(attribute_list)
    return rate

In [32]:
# random walk sampling
def random_walk_attribute_cover_rate(graph, ACR, attribute_list, attribute_bucket):
    startnode = rdm.sample(graph.nodes(),1)[0]
    currentnode = startnode
    iteration = 0 
    last_number_of_nodes = 0
    nodelist = set()
    total_iteration = 0
    acr = 0
    sampled_attributes = set()
    while acr < ACR:
        total_iteration += 1
        if currentnode not in nodelist:
            nodelist.add(currentnode)
            if currentnode in attribute_list.keys():
                for i in attribute_list[currentnode]:
                    if i not in sampled_attributes:
                        sampled_attributes.add(i)
                        acr += len(attribute_bucket[i])/nx.number_of_nodes(graph)
            
            
        # 15% probability jump back to startnode
        x = rdm.randint(1,100)
        if x < 16:
            currentnode = startnode
            
        # add edge when currentnode is not a sink
        if len(list(graph[currentnode])) > 0:
            nextnode = rdm.sample(list(graph[currentnode]),1)[0]
            currentnode = nextnode
        # jump back to the start node when reach a sink    
        else:
            currentnode = startnode
            
        # find a new startnode if number of nodes in sample does not grow
        if iteration < 10:
            iteration += 1
        else:
            if last_number_of_nodes == len(nodelist):
                startnode = rdm.sample(graph.nodes(),1)[0]
                currentnode = startnode
            iteration = 0
            last_number_of_nodes = len(nodelist)
    return total_iteration

Sampled chance via Degree

In [33]:
def sampled_chance_via_degree(graph, samples_path, numbers):
    sample = []
    result = {}
    for i in range(1,numbers+1):
        sample.append(set(nx.read_adjlist(samples_path + str(i) +".adjlist.gz", create_using=nx.DiGraph()).nodes))
    for n in graph.nodes:
        count = 0
        for i in range(0,numbers):
            if n in sample[i]:
                count+=1
        if graph.in_degree[n] not in result.keys():
            result[graph.in_degree[n]] = [count/numbers]
        else:
            result[graph.in_degree[n]].append(count/numbers)
        
    result2 = {}
    for i in result.keys():
        result2[i] = sum(result[i])/len(result[i])
    return dict(sorted(result2.items()))

Measuring Duplicates

In [34]:
# measuring covarance between any pair of samples
def duplicates_among_samples(path, numbers):
    sample = []
    for i in range(1,numbers+1):
        sample.append(nx.read_adjlist(path + str(i) +".adjlist.gz"))
    result = []
    n_nodes = len(sample[0].nodes)
    for i in range(0,numbers):
        for j in range(i+1, numbers):
            a = set(sample[i].nodes)
            b = set(sample[j].nodes)
            result.append(2-len(a|b)/n_nodes)
    return result

Utilities

In [43]:
# Save distribtuions into file
def save_distribution(distribution, filename):
    with open(filename, 'w') as f:
        for k in distribution.keys():
            f.write("%s\t%s\n" % (k, distribution[k]))

In [44]:
# Load distribtuions from file
def load_distribution(path):
    distribution = {}
    with open(path) as f:
        lines = f.readlines()
        for line in lines:
            if line[0] == '#':
                continue
            item = line.strip().split('\t')
            distribution[int(item[0])] = float(item[1])
    return distribution

In [45]:
# JS-divergence of two distribtuions
def JS_divergence(d1,d2):
    dit_d1 = dit.ScalarDistribution(list(d1.keys()), list(d1.values()))
    dit_d2 = dit.ScalarDistribution(list(d2.keys()), list(d2.values()))
    return jensen_shannon_divergence([dit_d1, dit_d2])

In [46]:
# another implementaion of js-divergence
def JSD(P, Q):
    Pv = list(P.values())
    Qv = list(Q.values())
    _P = Pv / norm(Pv, ord=1)
    _Q = Qv / norm(Qv, ord=1)
    _M = 0.5 * (_P + _Q)
    return 0.5 * (entropy(_P, _M) + entropy(_Q, _M))

In [84]:
# transfer adjlist to edgelist for plothop in snap.py
def adjlist_to_edgelist(path_subgraphs):
    # get all files from path_subgraphs
    list_subgraphs = listdir(path_subgraphs)
    for sg in list_subgraphs:
        #print(sg)
        #print(path_subgraphs + sg)
        #subgraph = nx.read_adjlist(path_subgraphs + sg, create_using=nx.DiGraph())
        subgraph = nx.read_adjlist(path_subgraphs + sg)
        subgraph_name = re.search('(.+?)\.adjlist\.gz', sg).group(1)        
        nx.write_edgelist(subgraph, path_subgraphs + subgraph_name + ".edgelist")

In [157]:
# Undirected graph
# do all evaluations at one time, and save distribtuions
def do_all_evaluations_udg(graph, path_subgraphs):
    # evaluations for graph
    degree_distribution_g = degree_distribution(graph)
    save_distribution(degree_distribution_g, "degree_distribution_full")
    
    clustering_coefficient_distribution_g = clustering_coefficient_distribution(graph)
    save_distribution(clustering_coefficient_distribution_g, "clustering_coefficient_distribution_full")
    
    min_distance_distribution_g = read_plothop_distribution("hop.tab")
    
    cc_distribution_g = cc_distribution(graph)
    save_distribution(cc_distribution_g, "connected_components_distribution_full")
    
    FLSV_g, SV_g = get_SVD_distribution(graph, 100)
    save_distribution(FLSV_g, "FLSV_full")
    save_distribution(SV_g, "SV_full")
    
    # get all files from path_subgraphs
    list_subgraphs = listdir(path_subgraphs)
    
    # JS divergence for different methods
    JS_degree = []
    MAE_clustering = []
    JS_mindistance = []
    JS_cc = []
    #JS_FLSV = []
    JS_SV = []
    
    for sg in list_subgraphs:
        print(sg)
        subgraph = nx.read_adjlist(path_subgraphs + sg)
        subgraph_name = re.search('(.+?)\.adjlist\.gz', sg).group(1)
        
        degree_distribution_subg = degree_distribution(subgraph)
        save_distribution(degree_distribution_subg, "degree_distribution_" + subgraph_name)
        js_degree_distribution = JS_divergence(degree_distribution_g, degree_distribution_subg)
        JS_degree.append(js_degree_distribution)
        
        clustering_coefficient_distribution_subg = clustering_coefficient_distribution(subgraph)
        save_distribution(clustering_coefficient_distribution_g, "clustering_coefficient_distribution_" + subgraph_name)
        absolute_error_clustering_coefficient = average_absolute_distance(clustering_coefficient_distribution_g, clustering_coefficient_distribution_subg)
        MAE_clustering.append(absolute_error_clustering_coefficient)
        
        min_distance_distribution_subg = read_plothop_distribution("hop." + subgraph_name + ".tab")
        js_min_distance_distribution = JS_divergence(min_distance_distribution_g, min_distance_distribution_subg)
        JS_mindistance.append(js_min_distance_distribution)
        
        cc_distribution_subg = cc_distribution(subgraph)
        save_distribution(cc_distribution_subg, "connected_components_distribution_" + subgraph_name)
        js_cc_distribution = JS_divergence(cc_distribution_g, cc_distribution_subg)
        JS_cc.append(js_cc_distribution)
        
        FLSV_subg, SV_subg = get_SVD_distribution(subgraph, 100)
        save_distribution(FLSV_subg, "FLSV_" + subgraph_name)
        save_distribution(SV_subg, "SV_" + subgraph_name)
        #js_FLSV = JSD(sample_FLSV_distribution(FLSV_g, 0.1), FLSV_subg) # here JS_divergence gives bad normalization
        js_SV = JS_divergence(SV_g, SV_subg)
        #JS_FLSV.append(js_FLSV)
        JS_SV.append(js_SV)
    
    print("--------------------degree--------------------")
    print(JS_degree)
    print(sum(JS_degree[0:10])/10)
    print(sum(JS_degree[10:20])/10)
    print(sum(JS_degree[20:30])/10)
    print(sum(JS_degree[30:40])/10)
    print("--------------------clustering coefficient--------------------")
    print(MAE_clustering)
    print(sum(MAE_clustering[0:10])/10)
    print(sum(MAE_clustering[10:20])/10)
    print(sum(MAE_clustering[20:30])/10)
    print(sum(MAE_clustering[30:40])/10)
    print("--------------------min distance--------------------")
    print(JS_mindistance)
    print(sum(JS_mindistance[0:10])/10)
    print(sum(JS_mindistance[10:20])/10)
    print(sum(JS_mindistance[20:30])/10)
    print(sum(JS_mindistance[30:40])/10)
    print("--------------------connected components--------------------")
    print(JS_cc)
    print(sum(JS_cc[0:10])/10)
    print(sum(JS_cc[10:20])/10)
    print(sum(JS_cc[20:30])/10)
    print(sum(JS_cc[30:40])/10)
    '''
    print("--------------------FLSV--------------------")
    print(JS_FLSV)
    print(sum(JS_FLSV[0:10])/10)
    print(sum(JS_FLSV[10:20])/10)
    print(sum(JS_FLSV[20:30])/10)
    print(sum(JS_FLSV[30:40])/10)
    '''
    print("--------------------SV--------------------")
    print(JS_SV)
    print(sum(JS_SV[0:10])/10)
    print(sum(JS_SV[10:20])/10)
    print(sum(JS_SV[20:30])/10)
    print(sum(JS_SV[30:40])/10)

In [158]:
do_all_evaluations_udg(g, "Dataset/Amazon/RATS/")

--- 66.11482191085815 seconds ---
rwa_1.adjlist.gz
--- 5.094374418258667 seconds ---
rwa_10.adjlist.gz
--- 5.924156188964844 seconds ---
rwa_2.adjlist.gz
--- 4.67094349861145 seconds ---
rwa_3.adjlist.gz
--- 5.850487470626831 seconds ---
rwa_4.adjlist.gz
--- 5.848160982131958 seconds ---
rwa_5.adjlist.gz
--- 5.219184875488281 seconds ---
rwa_6.adjlist.gz
--- 4.691073656082153 seconds ---
rwa_7.adjlist.gz
--- 4.974052667617798 seconds ---
rwa_8.adjlist.gz
--- 5.4237542152404785 seconds ---
rwa_9.adjlist.gz
--- 5.0731360912323 seconds ---
--------------------degree--------------------
[0.04954893007415295, 0.03736560921341381, 0.05206248510942091, 0.036949909844233186, 0.05399349361814343, 0.035229917750017936, 0.04466005390411887, 0.03560760666211804, 0.02188586414756921, 0.017273992872789634]
0.0384577863195978
0.0
0.0
0.0
--------------------clustering coefficient--------------------
[0.030317978520353756, 0.028162268466548696, 0.02807355878902223, 0.024385187958539785, 0.025521961433

In [41]:
# Directed graph
# do all evaluations at one time, and save distribtuions
def do_all_evaluations_dg(graph, path_subgraphs):
    # evaluations for graph
    
    indegree_distribution_g = indegree_distribution(graph)
    save_distribution(indegree_distribution_g, "indegree_distribution_full")
    
    outdegree_distribution_g = outdegree_distribution(graph)
    save_distribution(outdegree_distribution_g, "outdegree_distribution_full")
    
    graph_ud = graph.to_undirected()
    clustering_coefficient_distribution_g = clustering_coefficient_distribution(graph_ud)
    save_distribution(clustering_coefficient_distribution_g, "clustering_coefficient_distribution_full")
    
    min_distance_distribution_g = read_plothop_distribution("hop.tab")
    
    wcc_distribution_g = wcc_distribution(graph)
    save_distribution(wcc_distribution_g, "weakly_connected_components_distribution_full")
    
    scc_distribution_g = scc_distribution(graph)
    save_distribution(scc_distribution_g, "strongly_connected_components_distribution_full")
    
    FLSV_g, SV_g = get_SVD_distribution(graph, 100)
    save_distribution(FLSV_g, "FLSV_full")
    save_distribution(SV_g, "SV_full")
    
    # get all files from path_subgraphs
    list_subgraphs = listdir(path_subgraphs)
    

    # JS divergence for different methods
    JS_indegree = []
    JS_outdegree = []
    MAE_clustering = []
    JS_mindistance = []
    JS_wcc = []
    JS_scc = []
    JS_FLSV = []
    JS_SV = []
    
    for sg in list_subgraphs:
        print(sg)
        subgraph = nx.read_adjlist(path_subgraphs + sg, create_using=nx.DiGraph())
        subgraph_name = re.search('(.+?)\.adjlist\.gz', sg).group(1)
        
        # in case program crashes
        '''
        indegree_distribution_subg = load_distribution("indegree_distribution_" + subgraph_name)
        outdegree_distribution_subg = load_distribution("outdegree_distribution_" + subgraph_name)
        clustering_coefficient_distribution_subg = load_distribution("clustering_coefficient_distribution_" + subgraph_name)
        wcc_distribution_subg = load_distribution("weakly_connected_components_distribution_" + subgraph_name)
        scc_distribution_subg = load_distribution("strongly_connected_components_distribution_" + subgraph_name)
        FLSV_subg = load_distribution("FLSV_" + subgraph_name)
        SV_subg = load_distribution("SV_" + subgraph_name)
        '''        
        
        indegree_distribution_subg = indegree_distribution(subgraph)
        save_distribution(indegree_distribution_subg, "indegree_distribution_" + subgraph_name)
        js_indegree_distribution = JS_divergence(indegree_distribution_g, indegree_distribution_subg)
        JS_indegree.append(js_indegree_distribution)
        
        outdegree_distribution_subg = outdegree_distribution(subgraph)
        save_distribution(outdegree_distribution_subg, "outdegree_distribution_" + subgraph_name)
        js_outdegree_distribution = JS_divergence(outdegree_distribution_g, outdegree_distribution_subg)
        JS_outdegree.append(js_outdegree_distribution)
        
        clustering_coefficient_distribution_subg = clustering_coefficient_distribution(subgraph.to_undirected())
        save_distribution(clustering_coefficient_distribution_subg, "clustering_coefficient_distribution_" + subgraph_name)
        absolute_error_clustering_coefficient = average_absolute_distance(clustering_coefficient_distribution_g, clustering_coefficient_distribution_subg)
        MAE_clustering.append(absolute_error_clustering_coefficient)
        
        min_distance_distribution_subg = read_plothop_distribution("hop." + subgraph_name + ".tab")
        js_min_distance_distribution = JS_divergence(min_distance_distribution_g, min_distance_distribution_subg)
        JS_mindistance.append(js_min_distance_distribution)
        
        wcc_distribution_subg = wcc_distribution(subgraph)
        save_distribution(wcc_distribution_subg, "weakly_connected_components_distribution_" + subgraph_name)
        js_wcc_distribution = JS_divergence(wcc_distribution_g, wcc_distribution_subg)
        JS_wcc.append(js_wcc_distribution)
        
        scc_distribution_subg = scc_distribution(subgraph)
        save_distribution(scc_distribution_subg, "strongly_connected_components_distribution_" + subgraph_name)
        js_scc_distribution = JS_divergence(scc_distribution_g, scc_distribution_subg)
        JS_scc.append(js_scc_distribution)
        
        FLSV_subg, SV_subg = get_SVD_distribution(subgraph, 100)
        save_distribution(FLSV_subg, "FLSV_" + subgraph_name)
        save_distribution(SV_subg, "SV_" + subgraph_name)
        js_FLSV = JSD(sample_FLSV_distribution(FLSV_g, 0.1), FLSV_subg) # here JS_divergence gives bad normalization
        js_SV = JS_divergence(SV_g, SV_subg)
        JS_FLSV.append(js_FLSV)
        JS_SV.append(js_SV)
                

    print("--------------------indegree--------------------")
    print(JS_indegree)
    print(sum(JS_indegree[0:10])/10)
    print(sum(JS_indegree[10:20])/10)
    print(sum(JS_indegree[20:30])/10)
    print(sum(JS_indegree[30:40])/10)
    print("--------------------outdegree--------------------")
    print(JS_outdegree)
    print(sum(JS_outdegree[0:10])/10)
    print(sum(JS_outdegree[10:20])/10)
    print(sum(JS_outdegree[20:30])/10)
    print(sum(JS_outdegree[30:40])/10)    
    print("--------------------clustering coefficient--------------------")
    print(MAE_clustering)
    print(sum(MAE_clustering[0:10])/10)
    print(sum(MAE_clustering[10:20])/10)
    print(sum(MAE_clustering[20:30])/10)
    print(sum(MAE_clustering[30:40])/10)    
    print("--------------------min distance--------------------")
    print(JS_mindistance)
    print(sum(JS_mindistance[0:10])/10)
    print(sum(JS_mindistance[10:20])/10)
    print(sum(JS_mindistance[20:30])/10)
    print(sum(JS_mindistance[30:40])/10)
    print("--------------------weakly connected components--------------------")
    print(JS_wcc)
    print(sum(JS_wcc[0:10])/10)
    print(sum(JS_wcc[10:20])/10)
    print(sum(JS_wcc[20:30])/10)
    print(sum(JS_wcc[30:40])/10)    
    print("--------------------strongly connected components--------------------")
    print(JS_scc)
    print(sum(JS_scc[0:10])/10)
    print(sum(JS_scc[10:20])/10)
    print(sum(JS_scc[20:30])/10)
    print(sum(JS_scc[30:40])/10)    
    print("--------------------FLSV--------------------")
    print(JS_FLSV)
    print(sum(JS_FLSV[0:10])/10)
    print(sum(JS_FLSV[10:20])/10)
    print(sum(JS_FLSV[20:30])/10)
    print(sum(JS_FLSV[30:40])/10)
    print("--------------------SV--------------------")
    print(JS_SV)
    print(sum(JS_SV[0:10])/10)
    print(sum(JS_SV[10:20])/10)
    print(sum(JS_SV[20:30])/10)
    print(sum(JS_SV[30:40])/10)
    

In [92]:
# plot distributions
def plot_distribution(distributions, legends, xaxis, yaxis, filename, islog):
    fig = plt.figure()
    ax = fig . add_subplot (1,1,1)
    ax.set_xlabel(xaxis)
    ax.set_ylabel(yaxis)
    if islog:
        ax . set_xscale ('log')
        ax . set_yscale ('log')
    for distri in distributions:
        ax . plot (distri.keys(), distri.values())
    
    ax.legend(legends, loc = 'best')
    #fig . savefig (filename)
    tikz_save(filename)

Experiments

In [250]:

'''g = load_graph("Dataset/DBLP/com-dblp.ungraph.txt", "Undirected")
print(nx.number_of_nodes(g))
print(nx.number_of_edges(g))'''

317080
1049866


In [118]:
g = powerlaw_degree_graph_generator(56000, 2.3, seed = 1)
print(nx.number_of_nodes(g), nx.number_of_edges(g))

50180 99166


In [119]:
#al, ab = build_attribute_list_and_bucket("Dataset/DBLP/com-dblp.all.cmty.txt", "communities")
start_time = time.time()
al, ab = generate_attribute_list_and_bucket(g)
print("--- %s seconds ---" % (time.time() - start_time))

--- 9.118407487869263 seconds ---


In [262]:
nx.write_adjlist(sub_g, "RATS_" + str(1) + ".adjlist.gz")

In [263]:
adjlist_to_edgelist("Dataset/DBLP/RATS/")

In [120]:
start_time = time.time()
sub_g_rats, it = random_walk_attribute_sampling(g, 0.1, al, ab, 0.001, seed = 1)
print("--- %s seconds ---" % (time.time() - start_time))
print(it)

start_time = time.time()
sub_g_mhrw, it = MH_random_walk_sampling(g, 0.1, seed = 1)
print("--- %s seconds ---" % (time.time() - start_time))
print(it)

start_time = time.time()
sub_g_rw, it = random_walk_with_restart_sampling(g, 0.1, 0.15, seed = 1)
print("--- %s seconds ---" % (time.time() - start_time))
print(it)

start_time = time.time()
sub_g_rn = random_node_sampling(g, 0.1, seed = 1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 4.869808673858643 seconds ---
5507
--- 83.80635285377502 seconds ---
65976
--- 4.9952392578125 seconds ---
19429
--- 0.05984163284301758 seconds ---


In [110]:
degree_full = degree_distribution(g)
js_degree_rats = JS_divergence(degree_full, degree_distribution(sub_g_rats))
js_degree_mhrw = JS_divergence(degree_full, degree_distribution(sub_g_mhrw))
js_degree_rw = JS_divergence(degree_full, degree_distribution(sub_g_rw))
js_degree_rn = JS_divergence(degree_full, degree_distribution(sub_g_rn))
print(js_degree_rats, js_degree_rw, js_degree_mhrw, js_degree_rn)

0.04691926807777591 0.1724558486863561 0.15549480712393837 0.4042771298701031


In [93]:
plot_distribution([degree_full, degree_distribution(sub_g_rats), degree_distribution(sub_g_rw), degree_distribution(sub_g_mhrw), degree_distribution(sub_g_rn)], ["full","RATS","RWR","MHRW","RN"], "degree", "distribution", "degree.tikz", True)

Please add the following lines to your LaTeX preamble:

\usepackage[utf8]{inputenc}
\usepackage{fontspec} % This line only for XeLaTeX and LuaLaTeX
\usepackage{pgfplots}
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter


In [112]:
clustering_full = clustering_coefficient_distribution(g)

In [113]:
absolute_error_clustering_coefficient_rats = average_absolute_distance(clustering_full, clustering_coefficient_distribution(sub_g_rats))
absolute_error_clustering_coefficient_mhrw = average_absolute_distance(clustering_full, clustering_coefficient_distribution(sub_g_mhrw))
absolute_error_clustering_coefficient_rw = average_absolute_distance(clustering_full, clustering_coefficient_distribution(sub_g_rw))
absolute_error_clustering_coefficient_rn = average_absolute_distance(clustering_full, clustering_coefficient_distribution(sub_g_rn))
print(absolute_error_clustering_coefficient_rats,absolute_error_clustering_coefficient_mhrw,absolute_error_clustering_coefficient_rw,absolute_error_clustering_coefficient_rn)

0.21172978376712018 0.13176390689847942 0.1589862001637943 0.15291736550996116


In [94]:
plot_distribution([clustering_full, clustering_coefficient_distribution(sub_g_rats), clustering_coefficient_distribution(sub_g_rw), clustering_coefficient_distribution(sub_g_mhrw),  clustering_coefficient_distribution(sub_g_rn)], ["full","RATS","RWR","MHRW","RN"], "Degree", "Clustering Coefficient", "Power25_Clustering.tikz", True)

Please add the following lines to your LaTeX preamble:

\usepackage[utf8]{inputenc}
\usepackage{fontspec} % This line only for XeLaTeX and LuaLaTeX
\usepackage{pgfplots}
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter


In [114]:
cc_full = cc_distribution(g)
js_cc_rats = JS_divergence(cc_full, cc_distribution(sub_g_rats))
js_cc_mhrw = JS_divergence(cc_full, cc_distribution(sub_g_mhrw))
js_cc_rw = JS_divergence(cc_full, cc_distribution(sub_g_rw))
js_cc_rn = JS_divergence(cc_full, cc_distribution(sub_g_rn))
print(js_cc_rats, js_cc_mhrw, js_cc_rw, js_cc_rn)

0.0 0.0003005459183324684 0.0 0.3765244262326446


In [121]:
nx.write_edgelist(g, "power2_3.edgelist")
nx.write_edgelist(sub_g_rats, "power2_3_rats.edgelist")
nx.write_edgelist(sub_g_mhrw, "power2_3_mhrw.edgelist")
nx.write_edgelist(sub_g_rw, "power2_3_rw.edgelist")
nx.write_edgelist(sub_g_rn, "power2_3_rn.edgelist")

In [122]:
min_distance_distribution_g = read_plothop_distribution("hop.full.tab")
js_min_distance_rats = JS_divergence(min_distance_distribution_g, read_plothop_distribution("hop.rats.tab"))
js_min_distance_mhrw = JS_divergence(min_distance_distribution_g, read_plothop_distribution("hop.mhrw.tab"))
js_min_distance_rw = JS_divergence(min_distance_distribution_g, read_plothop_distribution("hop.rw.tab"))
js_min_distance_rn = JS_divergence(min_distance_distribution_g, read_plothop_distribution("hop.rn.tab"))
print(js_min_distance_rats, js_min_distance_mhrw, js_min_distance_rw, js_min_distance_rn)

0.13030241435170442 0.020555129280529272 0.12994992042756603 0.06855161571839874


In [270]:
degree_rats = clustering_coefficient_distribution(sub_g_rats)
degree_rw = clustering_coefficient_distribution(sub_g_rw)
degree_mhrw = clustering_coefficient_distribution(sub_g_mhrw)
degree_rn = clustering_coefficient_distribution(sub_g_rn)
degree_full = load_distribution("ijcai/degree_distribution_full")

In [275]:
c_rats = clustering_coefficient_distribution(sub_g_rats)
c_rw = clustering_coefficient_distribution(sub_g_rw)
c_mhrw = clustering_coefficient_distribution(sub_g_mhrw)
c_rn = clustering_coefficient_distribution(sub_g_rn)
c_full = load_distribution("ijcai/clustering_coefficient_distribution_full")
plot_distribution([c_rats, c_rw, c_mhrw, c_rn, c_full], ["RATS","RWR","MHRW","RN","full"], "degree", "clustering coefficient", "clustering.tikz", True)

Please add the following lines to your LaTeX preamble:

\usepackage[utf8]{inputenc}
\usepackage{fontspec} % This line only for XeLaTeX and LuaLaTeX
\usepackage{pgfplots}
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter


In [271]:
plot_distribution([degree_rats, degree_rw, degree_mhrw, degree_rn, degree_full], ["RATS","RWR","MHRW","RN","full"], "degree", "distribution", "degree.tikz", True)

Please add the following lines to your LaTeX preamble:

\usepackage[utf8]{inputenc}
\usepackage{fontspec} % This line only for XeLaTeX and LuaLaTeX
\usepackage{pgfplots}
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter


In [231]:
def do_all_evaluations_ijcai(subgraph, path_ff, subgraph_name):
    # load distributions
    degree_distribution_g = load_distribution(path_ff + "degree_distribution_full")
    clustering_coefficient_distribution_g = load_distribution(path_ff + "clustering_coefficient_distribution_full")
    cc_distribution_g = load_distribution(path_ff + "connected_components_distribution_full")
    min_distance_distribution_g = read_plothop_distribution(path_ff + "hop.tab")
    FLSV_g = load_distribution(path_ff + "FLSV_full")
    SV_g = load_distribution(path_ff + "SV_full")
    
    degree_distribution_subg = degree_distribution(subgraph)
    save_distribution(degree_distribution_subg, "degree_distribution_" + subgraph_name)
    js_degree_distribution = JS_divergence(degree_distribution_g, degree_distribution_subg)
    print("---------------degree---------------")
    print(js_degree_distribution)
        
    clustering_coefficient_distribution_subg = clustering_coefficient_distribution(subgraph)
    save_distribution(clustering_coefficient_distribution_subg, "clustering_coefficient_distribution_" + subgraph_name)
    absolute_error_clustering_coefficient = average_absolute_distance(clustering_coefficient_distribution_g, clustering_coefficient_distribution_subg)
    print("---------------clustering coefficient---------------")
    print(absolute_error_clustering_coefficient)
        
    min_distance_distribution_subg = read_plothop_distribution("hop." + subgraph_name + ".tab")
    js_min_distance_distribution = JS_divergence(min_distance_distribution_g, min_distance_distribution_subg)
    print("---------------min_distance---------------")
    print(js_min_distance_distribution)
        
    cc_distribution_subg = cc_distribution(subgraph)
    save_distribution(cc_distribution_subg, "connected_components_distribution_" + subgraph_name)
    js_cc_distribution = JS_divergence(cc_distribution_g, cc_distribution_subg)
    print("---------------cc---------------")
    print(js_cc_distribution)
    
    FLSV_subg, SV_subg = get_SVD_distribution(subgraph, 100)
    #save_distribution(FLSV_subg, "FLSV_" + subgraph_name)
    save_distribution(SV_subg, "SV_" + subgraph_name)
    #js_FLSV = JSD(sample_FLSV_distribution(FLSV_g, 0.1), FLSV_subg) # here JS_divergence gives bad normalization
    js_SV = JS_divergence(SV_g, SV_subg)
    #print("---------------FLSV---------------")
    #print(js_FLSV)
    print("---------------SV---------------")
    print(js_SV)

In [168]:
def do_all_evaluations_ijcai_di(subgraph, path_ff, subgraph_name):
    # load distributions
    indegree_distribution_g = load_distribution(path_ff + "indegree_distribution_full")
    outdegree_distribution_g = load_distribution(path_ff + "outdegree_distribution_full")
    clustering_coefficient_distribution_g = load_distribution(path_ff + "clustering_coefficient_distribution_full")
    wcc_distribution_g = load_distribution(path_ff + "weakly_connected_components_distribution_full")
    min_distance_distribution_g = read_plothop_distribution(path_ff + "hop.tab")
    #FLSV_g = load_distribution(path_ff + "FLSV_full")
    SV_g = load_distribution(path_ff + "SV_full")
    
    indegree_distribution_subg = indegree_distribution(subgraph)
    save_distribution(indegree_distribution_subg, "indegree_distribution_" + subgraph_name)
    js_indegree_distribution = JS_divergence(indegree_distribution_g, indegree_distribution_subg)
    print("---------------indegree---------------")
    print(js_indegree_distribution)
    
    outdegree_distribution_subg = outdegree_distribution(subgraph)
    save_distribution(outdegree_distribution_subg, "outdegree_distribution_" + subgraph_name)
    js_outdegree_distribution = JS_divergence(outdegree_distribution_g, outdegree_distribution_subg)
    print("---------------outdegree---------------")
    print(js_outdegree_distribution)
        
    clustering_coefficient_distribution_subg = clustering_coefficient_distribution(subgraph.to_undirected())
    save_distribution(clustering_coefficient_distribution_g, "clustering_coefficient_distribution_" + subgraph_name)
    absolute_error_clustering_coefficient = average_absolute_distance(clustering_coefficient_distribution_g, clustering_coefficient_distribution_subg)
    print("---------------clustering coefficient---------------")
    print(absolute_error_clustering_coefficient)
        
    min_distance_distribution_subg = read_plothop_distribution("hop." + subgraph_name + ".tab")
    js_min_distance_distribution = JS_divergence(min_distance_distribution_g, min_distance_distribution_subg)
    print("---------------min_distance---------------")
    print(js_min_distance_distribution)
        
    wcc_distribution_subg = wcc_distribution(subgraph)
    save_distribution(wcc_distribution_subg, "weakly_connected_components_distribution_" + subgraph_name)
    js_wcc_distribution = JS_divergence(wcc_distribution_g, wcc_distribution_subg)
    print("---------------wcc---------------")
    print(js_wcc_distribution)
        
    FLSV_subg, SV_subg = get_SVD_distribution(subgraph, 100)
    #save_distribution(FLSV_subg, "FLSV_" + subgraph_name)
    save_distribution(SV_subg, "SV_" + subgraph_name)
    #js_FLSV = JSD(sample_FLSV_distribution(FLSV_g, 0.1), FLSV_subg) # here JS_divergence gives bad normalization
    js_SV = JS_divergence(SV_g, SV_subg)
    #print("---------------FLSV---------------")
    #print(js_FLSV)
    print("---------------SV---------------")
    print(js_SV)

In [264]:
do_all_evaluations_ijcai(sub_g, "Dataset/DBLP/results/", "RATS_1")

---------------degree---------------
0.009170601663867295
---------------clustering coefficient---------------
0.2015752716473219
---------------min_distance---------------
0.07242657554049492
---------------cc---------------
0.0
--- 5.759595632553101 seconds ---
---------------SV---------------
0.0009436313087025638


In [153]:
# repeat experiments 10 times with random seed 1 to 10
def rwa_10():
    spendtime = []
    iterations = []
    for i in range(1, 11):
        rdm.seed(i)
        start_time = time.time()
        sample_g_rwa, it_rwa = random_walk_attribute_sampling(g, 0.1, al, ab, 0.01)
        print("--- %s seconds ---" % (time.time() - start_time))
        spendtime.append(time.time() - start_time)
        iterations.append(it_rwa)        
        print(nx.number_of_nodes(sample_g_rwa))
        print(nx.number_of_edges(sample_g_rwa))
        
        nx.write_adjlist(sample_g_rwa, "rwa_" + str(i) + ".adjlist.gz")
        #print(it_rwa)
    print(spendtime)
    print(sum(spendtime)/10)
    print(iterations)
    print(sum(iterations)/10)

In [154]:
rwa_10()

--- 4.933488607406616 seconds ---
33677
69855
--- 7.266780376434326 seconds ---
33510
69606
--- 8.126790046691895 seconds ---
33574
73775
--- 4.513562917709351 seconds ---
33576
69966
--- 4.594079256057739 seconds ---
33682
74159
--- 4.17477011680603 seconds ---
33594
71678
--- 5.804047584533691 seconds ---
33648
73586
--- 5.85080885887146 seconds ---
33660
79870
--- 7.525046110153198 seconds ---
33497
82159
--- 6.0101237297058105 seconds ---
33595
69309
[4.933488607406616, 7.266780376434326, 8.126790046691895, 4.513562917709351, 4.594079256057739, 4.17477011680603, 5.804047584533691, 5.85080885887146, 7.525046110153198, 6.0101237297058105]
5.879949760437012
[764, 420, 723, 250, 513, 607, 238, 466, 320, 283]
458.4


In [80]:
# repeat experiments 10 times with random seed 1 to 10
def ff_10():
    spendtime = []
    iterations = []
    for i in range(1, 11):
        start_time = time.time()
        sample_g_rwa, it_rwa = forest_fire_sampling(g, 0.1, 0.15, seed = i)
        print("--- %s seconds ---" % (time.time() - start_time))
        spendtime.append(time.time() - start_time)
        iterations.append(it_rwa)        
        print(nx.number_of_nodes(sample_g_rwa))
        print(nx.number_of_edges(sample_g_rwa))
        
        nx.write_adjlist(sample_g_rwa, "ff_" + str(i) + ".adjlist.gz")
        #print(it_rwa)
    print(spendtime)
    print(sum(spendtime)/10)
    print(iterations)
    print(sum(iterations)/10)

In [62]:
# repeat experiments 10 times with random seed 1 to 10
def MHRW_10():
    spendtime = []
    iterations = []
    for i in range(1, 11):
        start_time = time.time()
        sample_g_MHRW, it_MHRW = MH_random_walk_sampling(g, 0.1, seed = i)
        print("--- %s seconds ---" % (time.time() - start_time))
        spendtime.append(time.time() - start_time)
        iterations.append(it_MHRW)        
        print(nx.number_of_nodes(sample_g_MHRW))
        print(nx.number_of_edges(sample_g_MHRW))
        
        nx.write_adjlist(sample_g_MHRW, "MHRW_" + str(i) + ".adjlist.gz")
        #print(it_rwa)
    print(spendtime)
    print(sum(spendtime)/10)
    print(iterations)
    print(sum(iterations)/10)

In [63]:
MHRW_10()

--- 10.709856033325195 seconds ---
33486
70274
--- 10.3797607421875 seconds ---
33486
69569
--- 10.211945295333862 seconds ---
33486
70029
--- 10.36857008934021 seconds ---
33486
70010
--- 11.006011009216309 seconds ---
33486
70489
--- 10.53023886680603 seconds ---
33486
71581
--- 10.257216453552246 seconds ---
33486
69767
--- 9.935564756393433 seconds ---
33486
69581
--- 10.161395072937012 seconds ---
33486
70687
--- 10.537580728530884 seconds ---
33486
70962
[10.709856033325195, 10.3797607421875, 10.211945295333862, 10.36857008934021, 11.006011009216309, 10.53023886680603, 10.257216453552246, 9.935564756393433, 10.161395072937012, 10.537580728530884]
10.409813904762268
[150979, 151188, 150061, 150817, 155102, 153258, 150793, 148563, 150518, 154549]
151582.8


In [99]:
# repeat experiments 10 times with random seed 1 to 10
def ff_10_di():
    spendtime = []
    iterations = []
    for i in range(1, 11):
        start_time = time.time()
        sample_g_rwa, it_rwa = directed_forest_fire_sampling(g, 0.1, 0.15, 0.3, seed = i)
        print("--- %s seconds ---" % (time.time() - start_time))
        spendtime.append(time.time() - start_time)
        iterations.append(it_rwa)        
        print(nx.number_of_nodes(sample_g_rwa))
        print(nx.number_of_edges(sample_g_rwa))
        
        nx.write_adjlist(sample_g_rwa, "ff_" + str(i) + ".adjlist.gz")
        #print(it_rwa)
    print(spendtime)
    print(sum(spendtime)/10)
    print(iterations)
    print(sum(iterations)/10)

In [107]:
ff_10_di()

--- 7.622659921646118 seconds ---
377476
2194729
--- 7.462141990661621 seconds ---
377476
2177245
--- 7.607639789581299 seconds ---
377476
2166383
--- 7.234398126602173 seconds ---
377476
2058027
--- 7.343960285186768 seconds ---
377476
2201303
--- 7.3477888107299805 seconds ---
377476
2299502
--- 7.453097343444824 seconds ---
377476
2205385
--- 7.437936782836914 seconds ---
377476
2092968
--- 7.557390928268433 seconds ---
377476
1926158
--- 7.386552095413208 seconds ---
377476
2257447
[7.622659921646118, 7.462141990661621, 7.607639789581299, 7.234398126602173, 7.343960285186768, 7.3477888107299805, 7.453097343444824, 7.437936782836914, 7.557390928268433, 7.386552095413208]
7.445356607437134
[377476, 377476, 377477, 377476, 377476, 377476, 377476, 377476, 377476, 377476]
377476.1


In [109]:
def do_all_evaluations_jiayu_Di(path_ff):
    # load distributions
    indegree_distribution_g = load_distribution(path_ff + "indegree_distribution_full")
    outdegree_distribution_g = load_distribution(path_ff + "outdegree_distribution_full")
    clustering_coefficient_distribution_g = load_distribution(path_ff + "clustering_coefficient_distribution_full")
    #scc_distribution_g = load_distribution(path_ff + "strongly_connected_components_distribution_full")
    wcc_distribution_g = load_distribution(path_ff + "weakly_connected_components_distribution_full")
     
    # get all files from path_subgraphs
    list_subgraphs = listdir(path_ff + "subgraphs/")
    
    # JS divergence for different methods
    JS_indegree = []
    JS_outdegree = []
    MAE_clustering = []
    #JS_scc = []
    JS_wcc = []
    
    for sg in list_subgraphs:
        print(sg)
        subgraph = nx.read_adjlist(path_ff + "subgraphs/" + sg, create_using=nx.DiGraph())
        subgraph_name = re.search('(.+?)\.adjlist\.gz', sg).group(1)
        
        indegree_distribution_subg = indegree_distribution(subgraph)
        save_distribution(indegree_distribution_subg, "indegree_distribution_" + subgraph_name)
        js_indegree_distribution = JS_divergence(indegree_distribution_g, indegree_distribution_subg)
        JS_indegree.append(js_indegree_distribution)
        
        outdegree_distribution_subg = outdegree_distribution(subgraph)
        save_distribution(outdegree_distribution_subg, "outdegree_distribution_" + subgraph_name)
        js_outdegree_distribution = JS_divergence(outdegree_distribution_g, outdegree_distribution_subg)
        JS_outdegree.append(js_outdegree_distribution)
        
        clustering_coefficient_distribution_subg = clustering_coefficient_distribution(subgraph.to_undirected())
        save_distribution(clustering_coefficient_distribution_subg, "clustering_coefficient_distribution_" + subgraph_name)
        absolute_error_clustering_coefficient = average_absolute_distance(clustering_coefficient_distribution_g, clustering_coefficient_distribution_subg)
        MAE_clustering.append(absolute_error_clustering_coefficient)
        '''
        scc_distribution_subg = scc_distribution(subgraph)
        save_distribution(scc_distribution_subg, "Strongly_connected_components_distribution_" + subgraph_name)
        js_scc_distribution = JS_divergence(scc_distribution_g, scc_distribution_subg)
        JS_scc.append(js_scc_distribution)
        '''
        wcc_distribution_subg = wcc_distribution(subgraph)
        save_distribution(wcc_distribution_subg, "weakly_connected_components_distribution_" + subgraph_name)
        js_wcc_distribution = JS_divergence(wcc_distribution_g, wcc_distribution_subg)
        JS_wcc.append(js_wcc_distribution)
        
    print("--------------------indegree--------------------")
    print(JS_indegree)
    print(sum(JS_indegree[0:1])/1)
    print("--------------------outdegree--------------------")
    print(JS_outdegree)
    print(sum(JS_outdegree[0:1])/1)
    print("--------------------clustering coefficient--------------------")
    print(MAE_clustering)
    print(sum(MAE_clustering[0:1])/1)
    '''
    print("--------------------s connected components--------------------")
    print(JS_scc)
    print(sum(JS_scc[0:1])/1)
    '''
    print("--------------------w connected components--------------------")
    print(JS_wcc)
    print(sum(JS_wcc[0:1])/1)

In [110]:
do_all_evaluations_jiayu_Di("../WWW19 Random Walk Attribute Sampling/Dataset/cit-Patents/ffresult/")

ff_1.adjlist.gz
--------------------indegree--------------------
[0.03693899391092437]
0.03693899391092437
--------------------outdegree--------------------
[0.08054901011870186]
0.08054901011870186
--------------------clustering coefficient--------------------
[0.01940639302448519]
0.01940639302448519
--------------------w connected components--------------------
[0.0014122529798967012]
0.0014122529798967012


In [66]:
def do_all_evaluations_jiayu(path_ff):
    # load distributions
    degree_distribution_g = load_distribution(path_ff + "degree_distribution_full")
    clustering_coefficient_distribution_g = load_distribution(path_ff + "clustering_coefficient_distribution_full")
    cc_distribution_g = load_distribution(path_ff + "connected_components_distribution_full")
     
    # get all files from path_subgraphs
    list_subgraphs = listdir(path_ff + "subgraphs/")
    
    # JS divergence for different methods
    JS_degree = []
    MAE_clustering = []
    JS_cc = []
    
    for sg in list_subgraphs:
        print(sg)
        subgraph = nx.read_adjlist(path_ff + "subgraphs/" + sg)
        subgraph_name = re.search('(.+?)\.adjlist\.gz', sg).group(1)
        
        degree_distribution_subg = degree_distribution(subgraph)
        save_distribution(degree_distribution_subg, "degree_distribution_" + subgraph_name)
        js_degree_distribution = JS_divergence(degree_distribution_g, degree_distribution_subg)
        JS_degree.append(js_degree_distribution)
        
        clustering_coefficient_distribution_subg = clustering_coefficient_distribution(subgraph)
        save_distribution(clustering_coefficient_distribution_subg, "clustering_coefficient_distribution_" + subgraph_name)
        absolute_error_clustering_coefficient = average_absolute_distance(clustering_coefficient_distribution_g, clustering_coefficient_distribution_subg)
        MAE_clustering.append(absolute_error_clustering_coefficient)
        
        cc_distribution_subg = cc_distribution(subgraph)
        save_distribution(cc_distribution_subg, "connected_components_distribution_" + subgraph_name)
        js_cc_distribution = JS_divergence(cc_distribution_g, cc_distribution_subg)
        JS_cc.append(js_cc_distribution)
        
    print("--------------------degree--------------------")
    print(JS_degree)
    print(sum(JS_degree[0:10])/10)
    print("--------------------clustering coefficient--------------------")
    print(MAE_clustering)
    print(sum(MAE_clustering[0:10])/10)
    print("--------------------connected components--------------------")
    print(JS_cc)
    print(sum(JS_cc[0:10])/10)

In [89]:
do_all_evaluations_jiayu("../WWW19 Random Walk Attribute Sampling/Dataset/DBLP/ffresult/")

ff_1.adjlist.gz
ff_10.adjlist.gz
ff_2.adjlist.gz
ff_3.adjlist.gz
ff_4.adjlist.gz
ff_5.adjlist.gz
ff_6.adjlist.gz
ff_7.adjlist.gz
ff_8.adjlist.gz
ff_9.adjlist.gz
--------------------degree--------------------
[0.025554809221114638, 0.026140535274700838, 0.025519772706450894, 0.03126645290484564, 0.027986437563074062, 0.023895187210118962, 0.022763609959794096, 0.024923236858656495, 0.027514082344845647, 0.02472520718722926]
0.026028933123083055
--------------------clustering coefficient--------------------
[0.1394002903888917, 0.1572611412287203, 0.15824907576821215, 0.1734308632963491, 0.15726442520181555, 0.13769409063003074, 0.15164522196857033, 0.13968883066969442, 0.16616436255700245, 0.14773172101086027]
0.1528530022720147
--------------------connected components--------------------
[0.0, 0.0, 0.0, 0.0, 0.0001103910279615049, 1.5769070503103522e-05, 1.5769070503103522e-05, 0.0, 0.0, 1.5769070503103522e-05]
1.5769823947081547e-05


In [74]:
full_degree_distribution = load_distribution("../WWW19 Random Walk Attribute Sampling/distribution/degree_distribution_full")
ds_degree_distribution = load_distribution("../WWW19 Random Walk Attribute Sampling/distribution/DS_Amazon_distribution/DS_degree_distribution")
ff_degree_distribution = load_distribution("../WWW19 Random Walk Attribute Sampling/distribution/FF_Amazon_distribution/FF_degree_distribution")
rn_degree_distribution = load_distribution("../WWW19 Random Walk Attribute Sampling/distribution/RN_Amazon_distribution/RN_degree_distribution")
rwr_degree_distribution = load_distribution("../WWW19 Random Walk Attribute Sampling/distribution/RWR_Amazon_distribution/RWR_degree_distribution")

In [75]:
plot_distribution([full_degree_distribution, ds_degree_distribution, ff_degree_distribution, rn_degree_distribution, rwr_degree_distribution], ["full","DS","FF","RN","RWR"], "degree", "distribution", "degree.tikz", True)

Please add the following lines to your LaTeX preamble:

\usepackage[utf8]{inputenc}
\usepackage{fontspec} % This line only for XeLaTeX and LuaLaTeX
\usepackage{pgfplots}
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter


In [76]:
full_clustering_distribution = load_distribution("../WWW19 Random Walk Attribute Sampling/distribution/clustering_coefficient_distribution_full")
ds_clustering_distribution = load_distribution("../WWW19 Random Walk Attribute Sampling/distribution/DS_Amazon_distribution/DS_clustering_coefficient_distribution")
ff_clustering_distribution = load_distribution("../WWW19 Random Walk Attribute Sampling/distribution/FF_Amazon_distribution/FF_clustering_coefficient_distribution")
rn_clustering_distribution = load_distribution("../WWW19 Random Walk Attribute Sampling/distribution/RN_Amazon_distribution/RN_clustering_coefficient_distribution")
rwr_clustering_distribution = load_distribution("../WWW19 Random Walk Attribute Sampling/distribution/RWR_Amazon_distribution/RWR_clustering_coefficient_distribution")

In [77]:
plot_distribution([full_clustering_distribution, ds_clustering_distribution, ff_clustering_distribution, rn_clustering_distribution, rwr_clustering_distribution], ["full","DS","FF","RN","RWR"], "degree", "clustering coefficient", "clustering_coefficient.tikz", True)

Please add the following lines to your LaTeX preamble:

\usepackage[utf8]{inputenc}
\usepackage{fontspec} % This line only for XeLaTeX and LuaLaTeX
\usepackage{pgfplots}
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter


In [78]:
full_cc_distribution = load_distribution("../WWW19 Random Walk Attribute Sampling/distribution/connected_components_distribution_full")
ds_cc_distribution = load_distribution("../WWW19 Random Walk Attribute Sampling/distribution/DS_Amazon_distribution/DS_cc_distribution_subg")
ff_cc_distribution = load_distribution("../WWW19 Random Walk Attribute Sampling/distribution/FF_Amazon_distribution/FF_cc_distribution_subg")
rn_cc_distribution = load_distribution("../WWW19 Random Walk Attribute Sampling/distribution/RN_Amazon_distribution/RN_cc_distribution_subg")
rwr_cc_distribution = load_distribution("../WWW19 Random Walk Attribute Sampling/distribution/RWR_Amazon_distribution/RWR_cc_distribution_subg")

In [79]:
plot_distribution([full_cc_distribution, ds_cc_distribution, ff_cc_distribution, rn_cc_distribution, rwr_cc_distribution], ["full","DS","FF","RN","RWR"], "connected components", "distribution", "plot_cc.tikz", True)

Please add the following lines to your LaTeX preamble:

\usepackage[utf8]{inputenc}
\usepackage{fontspec} % This line only for XeLaTeX and LuaLaTeX
\usepackage{pgfplots}
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter


c:\users\haotian\appdata\local\programs\python\python36\lib\site-packages\matplotlib\axes\_base.py:3152: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=1.0, right=1.0
  'left=%s, right=%s') % (left, right))
c:\users\haotian\appdata\local\programs\python\python36\lib\site-packages\matplotlib\axes\_base.py:3471: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=1.0, top=1.0
  'bottom=%s, top=%s') % (bottom, top))


In [38]:
sgsgsg, ti = MH_random_walk_sampling(g, 0.1)

In [41]:
print(nx.number_of_nodes(sgsgsg))
print(nx.number_of_edges(sgsgsg))

399796
2275915


In [42]:
degree_distribution_g1 = degree_distribution(g)
degree_distribution_subg1 = degree_distribution(sgsgsg)
js_degree_distribution = JS_divergence(degree_distribution_g1, degree_distribution_subg1)

In [43]:
print(js_degree_distribution)

0.0667109228141074


In [267]:
# 1st -- real attributes, 2nd -- fake attributes
start_time = time.time()
al, ab = build_attribute_list_and_bucket("Dataset/LiveJournal/com-lj.all.cmty.txt", "communities")
#al, ab = generate_attribute_list_and_bucket(g)

print("--- %s seconds ---" % (time.time() - start_time))

--- 21.443650245666504 seconds ---


In [233]:
# facebook-100 use the 2nd one

# assortativity(g, al)
# assortativity(sample_g, fitst_attribute(al))

In [122]:
# caculate alpha
alpha = calculate_alpha(g, al, ab, 0.001)
print(alpha)

0.008669206277570207


In [23]:
def rw_10():
    spendtime = []
    iterations = []
    for i in range(1, 11):
        rdm.seed(i)
        start_time = time.time()
        sample_g_rw, it_rw = random_walk_sampling(g, 0.1)
        spendtime.append(time.time() - start_time)
        iterations.append(it_rw)
        print("--- %s seconds ---" % (time.time() - start_time))
        print(nx.number_of_nodes(sample_g_rw))
        print(nx.number_of_edges(sample_g_rw))
        
        nx.write_adjlist(sample_g_rw, "rw_" + str(i) + ".adjlist.gz")
        #print(it_rw)
    print(spendtime)
    print(sum(spendtime)/10)
    print(iterations)
    print(sum(iterations)/10)

In [24]:
def ra_10():
    spendtime = []
    for i in range(1, 11):
        rdm.seed(i)
        start_time = time.time()
        sample_g_ra = random_attribute_sampling(g, 0.1, al, ab, alpha)
        spendtime.append(time.time() - start_time)
        print("--- %s seconds ---" % (time.time() - start_time))
        print(nx.number_of_nodes(sample_g_ra))
        print(nx.number_of_edges(sample_g_ra))
        
        nx.write_adjlist(sample_g_ra, "ra_" + str(i) + ".adjlist.gz")
    print(spendtime)
    print(sum(spendtime)/10)

In [25]:
def rn_10():
    spendtime = []
    for i in range(1, 11):
        rdm.seed(i)
        start_time = time.time()
        sample_g_rn = random_node_sampling(g, 0.1)
        spendtime.append(time.time() - start_time)
        print("--- %s seconds ---" % (time.time() - start_time))
        print(nx.number_of_nodes(sample_g_rn))
        print(nx.number_of_edges(sample_g_rn))
        
        nx.write_adjlist(sample_g_rn, "rn_" + str(i) + ".adjlist.gz")
    print(spendtime)
    print(sum(spendtime)/10)

In [123]:
rwa_10()

--- 3.541752815246582 seconds ---
46502
36448
--- 2.1248695850372314 seconds ---
46502
37647
--- 3.1224608421325684 seconds ---
46502
44762
--- 6.355346918106079 seconds ---
46502
26850
--- 1.1389167308807373 seconds ---
46502
59557
--- 2.535579204559326 seconds ---
46502
38807
--- 5.062677383422852 seconds ---
46502
33684
--- 2.8125081062316895 seconds ---
46502
29029
--- 4.97372031211853 seconds ---
46502
27590
--- 4.5154969692230225 seconds ---
46502
29270
[3.541752815246582, 2.1248695850372314, 3.1224608421325684, 6.355346918106079, 1.1389167308807373, 2.535579204559326, 5.062677383422852, 2.8125081062316895, 4.97372031211853, 4.5154969692230225]
3.6183328866958617
[4454, 2461, 3198, 7195, 359, 3003, 6557, 4147, 6689, 5931]
4399.4


In [124]:
rw_10()

--- 324.8993136882782 seconds ---
46502
47410
--- 330.45741415023804 seconds ---
46502
44729
--- 348.1521461009979 seconds ---
46502
45149
--- 377.1786787509918 seconds ---
46502
37780
--- 374.17223167419434 seconds ---
46502
41555
--- 364.2274127006531 seconds ---
46502
38972
--- 367.2516658306122 seconds ---
46502
41233
--- 347.682918548584 seconds ---
46502
40846
--- 356.6663920879364 seconds ---
46502
41815
--- 344.10869097709656 seconds ---
46502
43644
[324.8993136882782, 330.45741415023804, 348.1521461009979, 377.1786787509918, 374.17223167419434, 364.2274127006531, 367.2516658306122, 347.682918548584, 356.6663920879364, 344.10869097709656]
353.47968645095824
[576302, 576544, 600876, 644909, 642280, 627545, 631301, 608446, 621528, 616971]
614670.2


In [125]:
ra_10()

--- 0.8638954162597656 seconds ---
46502
28506
--- 0.8627023696899414 seconds ---
46502
29771
--- 0.8870625495910645 seconds ---
46502
27404
--- 0.8714683055877686 seconds ---
46502
29409
--- 0.9036834239959717 seconds ---
46502
35774
--- 0.8751156330108643 seconds ---
46502
31876
--- 0.8482997417449951 seconds ---
46502
29301
--- 0.8594601154327393 seconds ---
46502
28435
--- 0.8858973979949951 seconds ---
46502
29923
--- 0.849846363067627 seconds ---
46502
25537
[0.8638954162597656, 0.8627023696899414, 0.8870625495910645, 0.8714683055877686, 0.9036834239959717, 0.8751156330108643, 0.8482997417449951, 0.8594601154327393, 0.8858973979949951, 0.849846363067627]
0.8707431316375732


In [126]:
rn_10()

--- 0.0624842643737793 seconds ---
46502
8471
--- 0.0624847412109375 seconds ---
46502
9265
--- 0.07810640335083008 seconds ---
46502
9141
--- 0.07810592651367188 seconds ---
46502
9290
--- 0.07285475730895996 seconds ---
46502
8321
--- 0.07810664176940918 seconds ---
46502
8228
--- 0.06246137619018555 seconds ---
46502
7615
--- 0.0781090259552002 seconds ---
46502
8659
--- 0.0780785083770752 seconds ---
46502
8419
--- 0.07812666893005371 seconds ---
46502
9147
[0.0624842643737793, 0.0624847412109375, 0.07810640335083008, 0.07810592651367188, 0.07285475730895996, 0.07810664176940918, 0.06246137619018555, 0.0781090259552002, 0.0780785083770752, 0.07812666893005371]
0.07289183139801025


In [248]:
adjlist_to_edgelist("Dataset/_nonattri_FourSquare/subgraphs/")

In [251]:
do_all_evaluations_udg(g, "Dataset/_nonattri_FourSquare/subgraphs/")

ra_1.adjlist.gz
ra_10.adjlist.gz
ra_2.adjlist.gz
ra_3.adjlist.gz
ra_4.adjlist.gz
ra_5.adjlist.gz
ra_6.adjlist.gz
ra_7.adjlist.gz
ra_8.adjlist.gz
ra_9.adjlist.gz
rn_1.adjlist.gz
rn_10.adjlist.gz
rn_2.adjlist.gz
rn_3.adjlist.gz
rn_4.adjlist.gz
rn_5.adjlist.gz
rn_6.adjlist.gz
rn_7.adjlist.gz
rn_8.adjlist.gz
rn_9.adjlist.gz
rwa_1.adjlist.gz
rwa_10.adjlist.gz
rwa_2.adjlist.gz
rwa_3.adjlist.gz
rwa_4.adjlist.gz
rwa_5.adjlist.gz
rwa_6.adjlist.gz
rwa_7.adjlist.gz
rwa_8.adjlist.gz
rwa_9.adjlist.gz
rw_1.adjlist.gz
rw_10.adjlist.gz
rw_2.adjlist.gz
rw_3.adjlist.gz
rw_4.adjlist.gz
rw_5.adjlist.gz
rw_6.adjlist.gz
rw_7.adjlist.gz
rw_8.adjlist.gz
rw_9.adjlist.gz
--------------------degree--------------------
[0.45818194786778665, 0.4664760753905184, 0.46923931581319644, 0.46952129758774275, 0.46698066758723433, 0.4762869433372674, 0.4733060583372337, 0.47206912745432383, 0.47493022959312237, 0.4868098754852803, 0.5294999521963408, 0.5252770044149981, 0.5249873796623326, 0.548438490941737, 0.54464431492